In [1]:
from fastai import *
from fastai.vision.all import *
from IPython.display import clear_output, DisplayHandle

print(torch.cuda.is_available())

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)
DisplayHandle.update = update_patch

True


In [3]:
def n_codes(fnames, is_partial=True):
  "Gather the codes from a list of `fnames`, full file paths"
  vals = set()
  if is_partial:
    random.shuffle(fnames)
    fnames = fnames[:10]
  for fname in fnames:
    msk = np.array(PILMask.create(fname))
    for val in np.unique(msk):
      if val not in vals:
        vals.add(val)
  vals = list(vals)
  p2c = dict()
  for i, val in enumerate(vals):
    p2c[i] = vals[i]
  return p2c


def get_mask_file(image_file, p2c, base_name='testing_groundtruth', mask_paths=''):
    nums = re.findall(string=image_file.name, pattern=r"\d+_\d+_\d+")[0]  # remove from list
    str_name = f'{base_name}_'+nums + image_file.suffix
    mask_path = (mask_paths/str_name)
    msk = np.array(PILMask.create(mask_path))
    mx = np.max(msk)
    for i, val in enumerate(p2c):
        # print(i, val)
        msk[msk == p2c[i]] = val
    return PILMask.create(msk)


def get_y(o):
    return get_mask_file(o, p2c)


# p2c = n_codes(test_mask_files)


In [4]:
# fine tune model on glomeruli data
model_path = '/home/ncamarda/coding/endotheliosisQuantifier/segmentation_model_dir/dynamic_unet_seg_model-e50_b16_v2.pkl'
# note we load v2 because this is also trained on an additional set of data

segmenter = load_learner(model_path)
